In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from typing import Tuple, Optional, Dict
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage
import wandb

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from research.data.natural_scenes import (
    NaturalScenesDataset
)
from research.models.components_2d import BlurConvTranspose2d
from research.models.fmri_decoders import VariationalDecoder, SpatialDecoder, SpatialDiscriminator, Decoder
from research.metrics.loss_functions import (
    EuclideanLoss, 
    EmbeddingClassifierLoss,
    ProbabalisticCrossEntropyLoss,
    VariationalLoss,
    CosineSimilarityLoss,
    EmbeddingDistributionLoss,
    ContrastiveDistanceLoss,
)
from research.experiments.nsd_decoding import NSDExperiment
from research.metrics.metrics import (
    cosine_similarity, 
    r2_score,
    pearsonr, 
    embedding_distance,
    cosine_distance,
    squared_euclidean_distance,
    contrastive_score,
    two_versus_two,
    smooth_euclidean_distance,
)

OSError: [WinError 127] The specified procedure could not be found. Error loading "C:\Users\Cefir\anaconda3\envs\Neurophysiological-Data-Decoding\lib\site-packages\torch\lib\torch_cuda_cpp.dll" or one of its dependencies.

In [ ]:
dataset_path = Path('D:\\Datasets\\NSD\\')
dataset = NaturalScenesDataset(dataset_path)
dataset_params = dict(
    subject_name='subj01',
    model_name='ViT-B=32',
    embedding_name='transformer.resblocks.0',
    encoder_name='fracridge',
    split_name='split-01',
    num_voxels=2500,
    normalize_X=True,
    normalize_Y=False,
)
train, val, test = dataset.load_data(**dataset_params)

In [ ]:
def run_experiment(
        embedding_name: str, 
        batch_size: int = 128,
        group: str = None,
        max_iterations: int = 10001,
        notes: str = None
):
    cosine_embedding = embedding_name == 'embedding'
    dataset_path = Path('D:\\Datasets\\NSD\\')
    dataset = NaturalScenesDataset(dataset_path)
    dataset_params = dict(
        subject_name='subj01',
        model_name='ViT-B=32',
        embedding_name=embedding_name,
        encoder_name='fracridge',
        split_name='split-01',
        num_voxels=2500,
        normalize_X=True,
        normalize_Y=(not cosine_embedding),
    )
    train, val, test = dataset.load_data(**dataset_params)
    
    model_params = dict(
        hidden_size=5000,
        dropout_p=0.9,
        normalize=cosine_embedding,
    )
    model = Decoder(num_voxels=dataset_params['num_voxels'], out_size=train.Y.shape[1], **model_params)
    
    criterion_params = dict(distance_metric=cosine_distance if cosine_embedding else squared_euclidean_distance)
    criterion = ContrastiveDistanceLoss(**criterion_params)
    
    #criterion_params = dict()
    #criterion = CosineSimilarityLoss() if cosine_embedding else nn.MSELoss()

    optimizer_params = dict(lr=1e-4)
    optimizer = Adam(
        params=[*model.parameters(), *criterion.parameters()],
        **optimizer_params,
    )

    device = torch.device('cuda')
    model.to(device)
    
    training_params = dict(
        train_on_averaged=False,
        batch_size=batch_size,
        evaluation_interval=2500,
        evaluation_subset_size=500,
    )
    experiment = NSDExperiment(train, val, test, device, model, criterion, optimizer, **training_params)
    
    config = {
        **dataset_params,
        **training_params,
        'model': model,
        **model_params,
        'criterion': criterion,
        **criterion_params,
        'optimizer': optimizer,
        **optimizer_params,
    }
    wandb.init(project='natural-scenes', config=config, group=group, notes=notes)
    wandb.define_metric("*", summary="max")
    wandb.define_metric("*", summary="min")
    
    experiment.train_model(max_iterations=max_iterations, logger=wandb.log)
    
    return experiment

def save_experiment(experiment):
    decoded_features_path = Path('D:\\Datasets\\NSD\\derivatives\\decoded_features\\')
    save_file_name = wandb.run.group if wandb.run.group else wandb.run.name
    save_file_path = decoded_features_path / wandb.config['model_name'] / f'{save_file_name}.hdf5'
    h5_key = (wandb.config['subject_name'], wandb.config['embedding_name'])
    attributes = dict(wandb.config)
    attributes['wandb_run_name'] = wandb.run.name
    attributes['wandb_run_url'] = wandb.run.url
    attributes['wandb_group'] = wandb.run.group
    attributes['wandb_notes'] = wandb.run.notes
    experiment.save(save_file_path, attributes, h5_key)

In [ ]:
shared_params = {
    'group': 'group-1',
    #'notes': 'Contrastive loss -> mean distance loss. normalize_y for resblocks.',
    'max_iterations': 10001,
}

In [ ]:
experiment = run_experiment('embedding', **shared_params)
save_experiment(experiment)
wandb.finish()

In [ ]:
embedding_names = [f'transformer.resblocks.{i}' for i in range(6, 12)]
for embedding_name in embedding_names:
    experiment = None
    gc.collect()
    torch.cuda.empty_cache()
    experiment = run_experiment(embedding_name, batch_size=24, **shared_params)
    save_experiment(experiment)
    wandb.finish()

In [ ]:
#wandb.finish()
experiment = None
gc.collect()
torch.cuda.empty_cache()

In [ ]:
sweep_config = {
    'name': 'clip-embeddings',
    'method': 'grid',
    'parameters': {
        'embedding_name': {
            'values': [f'transformer.resblocks.{i}' for i in range(12)]
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

In [ ]:
experiment.train_model(max_iterations=3500, logger=wandb.log)

In [ ]:
save_path = 'D:/Datasets/NSD/derivatives/decoded_features/'


In [ ]:
api = wandb.Api()
runs = api.runs('efirdc/natural-scenes') 

In [ ]:
for run in runs:
    print(run.history())

In [ ]:
run = runs[0]
run.summary['_step']

In [ ]:
t = time.perf_counter()
experiment.evaluate()
print(time.perf_counter() - t)

In [ ]:
from typing import Tuple, Optional, Dict
from functools import partial

import wandb
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from research.models.components_2d import BlurConvTranspose2d
from research.models.fmri_decoders import VariationalDecoder, SpatialDecoder, SpatialDiscriminator, Decoder
from research.metrics.loss_functions import (
    EuclideanLoss, 
    EmbeddingClassifierLoss,
    ProbabalisticCrossEntropyLoss,
    VariationalLoss,
    CosineSimilarityLoss,
    EmbeddingDistributionLoss,
    ContrastiveDistanceLoss,
)

from research.metrics.metrics import (
    cosine_similarity, 
    r2_score,
    pearsonr, 
    embedding_distance,
    cosine_distance,
    squared_euclidean_distance,
    contrastive_score,
    two_versus_two,
    smooth_euclidean_distance,
)

#model = VariationalDecoder(
#    num_voxels, out_size, hidden_size, 
#    #decoder_class=decoder_class,
#    #decoder_params=decoder_params,
#)


model_params = dict(
    hidden_size=5000,
    dropout_p=0.9,
)
model = Decoder(num_voxels=dataset_params['num_voxels'], out_size=train.Y.shape[1], **model_params)

training_params = dict(train_on_averaged=False)
#criterion = ContrastiveDistanceLoss(distance_metric=partial(smooth_euclidean_distance, beta=0.25))

#criterion = nn.MSELoss()
#criterion = CosineSimilarityLoss()
#criterion = nn.L1Loss()
#criterion = VariationalLoss(distance_loss=criterion, kld_weight=training_params['kld_weight'])

criterion_params = dict(distance_metric=cosine_distance)
criterion = ContrastiveDistanceLoss(**criterion_params)

optimizer_params = dict(lr=1e-4)
optimizer = Adam(
    params=[*model.parameters(), *criterion.parameters()],
    **optimizer_params,
)

device = torch.device('cuda')
model.to(device)

train_dataset = train.torch_dataset(training_params['train_on_averaged'])
dataloader = DataLoader(train_dataset, shuffle=True, batch_size=128)
def get_data_iterator(loader):
    while True:
        for batch in loader:
            yield batch
data_iterator = get_data_iterator(dataloader)

iteration = 0

def run_all(X):
    Y = []
    for x in X:
        y = model(x.to(device)[None])
        if isinstance(y, Tuple):
            y = y[0]
        Y.append(y)
    return torch.cat(Y).cpu()

evaluation_metrics = [r2_score, pearsonr]
distance_metrics = [cosine_distance, squared_euclidean_distance]
distance_classification_measures = [two_versus_two, contrastive_score]
def evaluate():
    evaluation_dict = {}
    
    model.eval()
    with torch.no_grad():
        Y_train_pred = run_all(train_dataset.tensors[0])
        Y_val_pred = run_all(val.X)
        Y_val_avg_pred = run_all(val.X_averaged)
        
    for evaluation_metric in evaluation_metrics:
        evaluation_dict[evaluation_metric.__name__] = {
            'train': evaluation_metric(train_dataset.tensors[1], Y_train_pred),
            'val': evaluation_metric(val.Y, Y_val_pred),
            'val_averaged': evaluation_metric(val.Y_averaged, Y_val_avg_pred),
        }

    for distance_metric in distance_metrics:
        distances = {}
        distances['train'] = distance_metric(train_dataset.tensors[1][None, :], Y_train_pred[:, None])
        distances['val'] = distance_metric(val.Y[None, :], Y_val_pred[:, None])
        distances['val_averaged'] = distance_metric(val.Y_averaged[None, :], Y_val_avg_pred[:, None])
        stats = {
            measure.__name__: {
                d_name: measure(d).item() for d_name, d in distances.items()
            }
            for measure in distance_classification_measures
        }
        stats.update({
            'mean': {d_name: d.diag().mean().item() for d_name, d in distances.items()},
            'std': {d_name: d.diag().std().item() for d_name, d in distances.items()},
        })
        evaluation_dict[distance_metric.__name__] = stats
    
    return evaluation_dict

wandb.init(
    project='natural-scenes',
    config={
        **dataset_params,
        **training_params,
        'model': model,
        **model_params,
        'criterion': criterion,
        **criterion_params,
        'optimizer': optimizer,
        **optimizer_params,
    },
)
wandb.define_metric("*", summary="max")
wandb.define_metric("*", summary="min")

print(model)

In [ ]:
max_iterations = 1500
for i in range(max_iterations):
    evaluation_dict = {}
    if iteration % 250 == 0:
        evaluation_dict = evaluate()
    
    x, y = next(data_iterator)
    x = x.to(device)
    y = y.to(device)

    model.train()
    model_out = model(x)
    if isinstance(model_out, Tuple):
        y_pred, mu, log_var = model_out
        loss, loss_dict = criterion(y, y_pred, mu, log_var)
    else:
        y_pred = model_out
        loss = criterion(y, y_pred)
        loss_dict = {'loss': loss}
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    
    log_dict = {**loss_dict, **evaluation_dict}
    wandb.log(log_dict)
    
    iteration += 1

In [ ]:
model.eval()
with torch.no_grad():
    Y_train_pred = run_all(train_dataset.tensors[0])
    Y_val_pred = run_all(val.X)
    Y_val_avg_pred = run_all(val.X_averaged)

for evaluation_metric in evaluation_metrics:
    evaluation_dict[evaluation_metric.__name__] = {
        'train': evaluation_metric(train_dataset.tensors[1], Y_train_pred),
        'val': evaluation_metric(val.Y, Y_val_pred),
        'val_averaged': evaluation_metric(val.Y_averaged, Y_val_avg_pred),
    }

In [ ]:
Y_val_pred.mean(dim=1)

In [ ]:
wandb.finish()

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from pathlib import Path

out_path = derivatives_path / 'decoded_features'

def run_all_variational(X):
    mean = []
    std = []
    for x in X:
        _, mu, log_var = model(x.to(device)[None])
        mean.append(mu)
        std.append(torch.exp(0.5 * log_var))
    mean = torch.cat(mean)
    std = torch.cat(std)
    out = torch.stack([mean, std], dim=1)
    return out.cpu().numpy()

with torch.no_grad():
    Y_val_pred = run_all_variational(X_val).cpu().numpy()
    Y_val_avg_pred = run_all_variational(X_val_avg).cpu().numpy()

version = '1-0'
code_name = 'averaging'
out_dir = out_path / embedding_model / embedding_key / subject_name
out_dir.mkdir(parents=True, exist_ok=True)

np.save(out_dir / f'Y_val_pred__{code_name}__v{version}.npy', Y_val_pred)
np.save(out_dir / f'Y_val__{code_name}__v{version}.npy', Y_val)

np.save(out_dir / f'Y_val_avg_pred__{code_name}__v{version}.npy', Y_val_avg_pred)
np.save(out_dir / f'Y_val_avg__{code_name}__v{version}.npy', Y_val_avg)

In [ ]:
from pathlib import Path

out_path = derivatives_path / 'decoded_features'

with torch.no_grad():
    Y_val_pred = run_all(X_val)
    Y_val_avg_pred = run_all(X_val_avg)

version = '1-0'
code_name = 'standard'
out_dir = out_path / embedding_model / embedding_key / subject_name
out_dir.mkdir(parents=True, exist_ok=True)

np.save(out_dir / f'Y_val_pred__{code_name}__v{version}.npy', Y_val_pred)
np.save(out_dir / f'Y_val__{code_name}__v{version}.npy', Y_val)

np.save(out_dir / f'Y_val_avg_pred__{code_name}__v{version}.npy', Y_val_avg_pred)
np.save(out_dir / f'Y_val_avg__{code_name}__v{version}.npy', Y_val_avg)

In [ ]:

num_samples = 4
Y_pred_mean = Y_val_pred[:, 0]
Y_pred_std = Y_val_pred[:, 1]
sample = np.random.randn(num_samples, *Y_pred_mean.shape)
Y_pred = Y_pred_mean[None] + Y_pred_std[None] * sample
Y_pred.shape

In [ ]:
Y_pred.std()

In [ ]:
distances = euclidean_distance(Y1[None, :], Y2[:, None])
different = distances + distances.T

distances_diag = torch.diag(distances)
same = distances_diag[None, :] + distances_diag[:, None]

comparison = same < different

plt.imshow(comparison)

In [ ]:
N = 50
distances = euclidean_distance(Y1[None, :], Y2[:, None])
column_score = ((distances < distances.diag()).sum(dim=0) / (N - 1)).mean()
row_score = ((distances < distances.diag()[:, None]).sum(dim=1) / (N - 1)).mean()
print(column_score, row_score)
plt.imshow(distances < distances.diag())

In [ ]:
def test_score(Y, Y_pred, dim=0, cast_dtype=torch.float64):
    Y = Y.to(torch.float64)
    Y_pred = Y_pred.to(torch.float64)

    Y = Y - Y.mean(dim=dim, keepdim=True)
    Y_pred = Y_pred - Y_pred.mean(dim=dim, keepdim=True)

    Y = Y / torch.norm(Y, dim=dim, keepdim=True)
    Y_pred = Y_pred / torch.norm(Y_pred, dim=dim, keepdim=True)

    r = (Y * Y_pred).sum(dim=dim)
    print(r)
    return r.mean().item()

test_score(val.Y, Y_val_pred / Y_val_pred.norm(dim=1, keepdim=True))

In [ ]:
print(val.Y.shape)
Y_norm = Y_val_pred / Y_val_pred.norm(dim=1, keepdim=True)

@interact(i=(0, 512), j=(0, 3000))
def show(i, j):
    x = np.arange(val.Y.shape[0])
    y = val.Y[:, i]
    y_pred = Y_norm[:, i]
    plt.figure(figsize=(24, 8))
    plt.plot(x[:j], y[:j])
    plt.plot(x[:j], y_pred[:j])

In [ ]:
constrastive_distance(Y1, Y2, distance_measure=euclidean_distance)